# =============================================================================
# Семинар 3. ИДС / NLP
# Частотный анализ повести Пушкина «Метель»
# Облака слов, POS-фильтрация, синтаксические зависимости
# =============================================================================

In [ ]:


# ─── 1. Импорты и настройки ──────────────────────────────────────────────────

import re
import string
from collections import Counter

import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-pastel')
%matplotlib inline

import nltk
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

from wordcloud import WordCloud

%pip install pymorphy3 -q
import pymorphy3

%pip install spacy ru_core_news_sm -q
import spacy
from spacy import displacy

print("Все необходимые библиотеки загружены\n")

# ─── 2. Чтение и начальная очистка текста ────────────────────────────────────

with open('pushkin-metel.txt', 'r', encoding='utf-8') as fh:
    original_text = fh.read()

print(f"Длина исходного текста: {len(original_text):,} символов")
print("Первые 220 символов:\n", original_text[:220], "...\n")

# Приведение к нижнему регистру + удаление пунктуации и цифр
clean_text = original_text.lower()
clean_text = re.sub(r'[^\w\sа-яё]', '', clean_text)          # всё кроме букв и пробелов
clean_text = re.sub(r'\s+', ' ', clean_text).strip()

print("Длина после базовой очистки:", len(clean_text))
print("Пример:", clean_text[:180], "...\n")

# ─── 3. Токенизация и частотный словарь (до стоп-слов) ───────────────────────

tokens_raw = word_tokenize(clean_text, language='russian')
print(f"Всего токенов: {len(tokens_raw):,}")

freq_raw = FreqDist(tokens_raw)

print("\nТоп-15 слов до удаления стоп-слов:")
for word, cnt in freq_raw.most_common(15):
    print(f"{word:>10}  {cnt:>6}")

plt.figure(figsize=(11, 5))
freq_raw.plot(30, title="Топ-30 слов (включая служебные)", cumulative=False)
plt.grid(True, alpha=0.3)
plt.show()

# ─── 4. Фильтрация стоп-слов ─────────────────────────────────────────────────

ru_stops = set(stopwords.words('russian'))
ru_stops.update(['это', 'было', 'стал', 'быть', 'который', 'тот', 'сам', 'всё', 'сказал'])

tokens_filtered = [t for t in tokens_raw if t not in ru_stops and len(t) > 2]

print(f"Токенов после фильтрации: {len(tokens_filtered):,}")

freq_filtered = FreqDist(tokens_filtered)

print("\nТоп-12 значимых слов:")
for w, c in freq_filtered.most_common(12):
    print(f"{w:>12} : {c:>5}")

plt.figure(figsize=(10, 4.5))
freq_filtered.plot(25, title="Топ-25 после удаления стоп-слов")
plt.show()

# ─── 5. Общее облако слов ────────────────────────────────────────────────────

text_for_cloud = ' '.join(tokens_filtered)

cloud_all = WordCloud(
    width=1000, height=600,
    background_color='linen',
    colormap='Dark2',
    max_words=200,
    min_font_size=10, max_font_size=180
).generate(text_for_cloud)

plt.figure(figsize=(14, 8))
plt.imshow(cloud_all, interpolation='bicubic')
plt.axis('off')
plt.title("Облако слов — «Метель» Пушкина (без стоп-слов)")
plt.show()

# ─── 6. POS-анализ и облака по частям речи (pymorphy3) ──────────────────────

morph = pymorphy3.MorphAnalyzer()

nouns, verbs, adjs = [], [], []

for token in tokens_filtered:
    p = morph.parse(token)[0]
    pos = p.tag.POS
    
    if pos == 'NOUN':
        nouns.append(p.normal_form)
    elif pos in ('VERB', 'INFN'):
        verbs.append(p.normal_form)
    elif pos in ('ADJF', 'ADJS'):
        adjs.append(p.normal_form)

print(f"Существительных: {len(nouns):,}")
print(f"Глаголов/инфинитивов: {len(verbs):,}")
print(f"Прилагательных: {len(adjs):,}\n")

noun_counter = Counter(nouns)
verb_counter = Counter(verbs)
adj_counter  = Counter(adjs)

# Топ по каждой категории
print("Топ-7 существительных:", noun_counter.most_common(7))
print("Топ-7 глаголов     :", verb_counter.most_common(7))
print("Топ-7 прилагательных:", adj_counter.most_common(7))

# Облака слов по частям речи
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

cloud_n = WordCloud(background_color='azure', colormap='Blues', width=600, height=400).generate_from_frequencies(noun_counter)
axes[0].imshow(cloud_n, interpolation='bilinear')
axes[0].set_title('Существительные')
axes[0].axis('off')

cloud_v = WordCloud(background_color='oldlace', colormap='OrRd', width=600, height=400).generate_from_frequencies(verb_counter)
axes[1].imshow(cloud_v, interpolation='bilinear')
axes[1].set_title('Глаголы')
axes[1].axis('off')

cloud_a = WordCloud(background_color='honeydew', colormap='Greens', width=600, height=400).generate_from_frequencies(adj_counter)
axes[2].imshow(cloud_a, interpolation='bilinear')
axes[2].set_title('Прилагательные')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# ─── 7. Синтаксические зависимости (spacy) ──────────────────────────────────

nlp = spacy.load("ru_core_news_sm")

# Берём первые 6 предложений из оригинального текста
doc_full = nlp(original_text)
first_sents = list(doc_full.sents)[:6]

print("Синтаксические зависимости — первые 6 предложений\n")

for i, sentence in enumerate(first_sents, 1):
    print(f"Предложение {i}:\n{sentence.text.strip()}\n")
    displacy.render(sentence, style="dep", jupyter=True,
                    options={'distance': 105, 'compact': True, 'bg': '#118ab2', 'color': '#ffffff'})
    print("─" * 90 + "\n")